In [3]:
!pip install tqdm --user


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 59 kB 2.9 MB/s eta 0:00:011
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
############ Generate Images ###################
import math
import numpy as np
import cv2
import cv2, os
import matplotlib.pyplot as plt
from tqdm import tqdm
import fnmatch

In [5]:
######## SPATIAL: read prediciton, ground truth and  mouse (x,y) from CSV ################

pred_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/prediction_XX_26180.csv').astype(np.int64)
pred_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/prediction_YY_26180.csv').astype(np.int64)

gt_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/GT_Fix_XX_26180.csv').astype(np.int64)
gt_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/GT_Fix_YY_26180.csv').astype(np.int64)

mouse_x_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/mouse_XX_26180.csv').astype(np.int64)
mouse_y_data = np.genfromtxt('/home/kkjk/LSTM/Co-ordinates_mouse_cursor_GT_prediction/mouse_YY_26180.csv').astype(np.int64)




In [6]:
################## SPATIAL ############################## 

def GenerateImages(Input, Imagename):
    """ 
    This function generates binary map/image of size 1200*1920 out of (x,y)co-ordinates
    
    Parameters:
        Input(stacked array of (x,y)): join (x,y) of prediction or ground truth or mouse
        Imagename(string): Name of the image to be created and saved
    
    Returns:
        path of created image. changes for spatial and temporal data. 
    """
    Op_Image = np.zeros((1200, 1920), dtype=np.int64)

    for x, y in zip(Input[0], Input[1]):
        # print('x,y', x, y)
        Op_Image[y][x] = 255
        op_map = np.float32(Op_Image)
    NewImage = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/'+Imagename
    cv2.imwrite(NewImage, op_map)
    return NewImage


In [7]:
## 3 test users with total of 39 interfaces, so 39*340 = 13260 (x,y) points
## UI/task split ups: 340 (x,y) points per task

k = 1

for i in range(0, 13260, 340):

    # print(i,i+340)
    pred_X_Y = np.vstack((pred_x_data[i:i+340], pred_y_data[i:i+340].T))
    gt_X_Y = np.vstack((gt_x_data[i:i+340], gt_y_data[i:i+340].T))
    mouse_X_Y = np.vstack((mouse_x_data[i:i+340], mouse_y_data[i:i+340].T))
    Pred_X = GenerateImages(pred_X_Y, 'predict_spatial_task_'+str(i)+'.png')
    True_X = GenerateImages(gt_X_Y, 'true_spatial_task_'+str(i)+'.png')
    Mouse_X = GenerateImages(mouse_X_Y, 'mouse_spatial_task_'+str(i)+'.png')
    k = k + 1
    print(k)
print(k, 'spatial images generated')

2
2 spatial images generated


In [8]:
#################### GENERATING HEATMAP: SPATIAL ###########################
def GaussianMask(sizex, sizey, sigma, center=None, fix=1):
    """
    This function performs gaussian convolution/masking
    Parameters:
        sizex  : mask width
        sizey  : mask height
        sigma  : gaussian standard deviation
        center : gaussian mean
        fix    : gaussian max
    returns:
        gaussian mask
    """

    x = np.arange(0, sizex, 1, float)
    y = np.arange(0, sizey, 1, float)
    x, y = np.meshgrid(x, y)

    if center is None:
        x0 = sizex // 2
        y0 = sizey // 2

    else:
        if np.isnan(center[0]) == False and np.isnan(center[1]) == False:
            x0 = center[0]
            y0 = center[1]

        else:
            return np.zeros((sizey, sizex))

    return fix * np.exp(-4 * np.log(2) * ((x - x0) ** 2 + (y - y0) ** 2) / sigma ** 2)


def Fixpos2Densemap(fix_arr, width, height): #, imgfile, alpha=0.5, threshold=10):
    """
    This function creates heatmap/attention map
    Parameters:
        fix_arr   : fixation array number of subjects x 3(x,y,fixation)
        width     : output image width
        height    : output image height
        imgfile   : image file (optional)
        alpha     : marge rate imgfile and heatmap (optional)
        threshold : heatmap threshold(0~255)
    returns:
        heatmap
    """
    W = width
    H = height
    heatmap = np.zeros((H, W), np.float32)
    for n_subject in tqdm(range(fix_arr.shape[0])):
        heatmap += GaussianMask(W, H, 64, (fix_arr[n_subject, 0], fix_arr[n_subject, 1]), fix_arr[n_subject, 2])

    # Normalization to have values between 0-1
    
    heatmap = heatmap / np.amax(heatmap)
    heatmap = heatmap * 255
    heatmap = heatmap.astype("uint8")
    # heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return heatmap

    if imgfile.any():
    # Resize heatmap to imgfile shape 
        h, w, _ = imgfile.shape
        heatmap = cv2.resize(heatmap, (w, h))
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
      
    # Create mask
        mask = np.where(heatmap<=threshold, 1, 0)
        mask = np.reshape(mask, (h, w, 1))
        mask = np.repeat(mask, 3, axis=2)
      
    # Marge images
        marge = imgfile*mask + heatmap_color*(1-mask)
        marge = marge.astype("uint8")
        marge = cv2.addWeighted(imgfile, 1-alpha, marge,alpha,0)
        print('Im with screenshot and color')

        return marge, mask, heatmap_color

    else:
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        return heatmap

In [9]:
# imdir for reading input and outdir for storing output heatmap




In [10]:
##### Generate heatmaps for prediction data ################
p_imdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_prediction/'

p_outdir =  '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/'

In [12]:
for i in range(0,13260,340):

    pfile = 'predict_spatial_task_'+str(i)+'.png'
  
    
    ppath = os.path.join(p_imdir, pfile)
    print('prediction:',ppath)
      
    img = cv2.imread(ppath, cv2.IMREAD_GRAYSCALE)
    result = np.where(img == 255)
#     print('result',result)
    # y = np.array(result[0])[::-1]
    y = np.array(result[0])
    x = np.array(result[1])
    z = np.ones(x.shape)
    fix_arr = np.column_stack([x, y, z])
    H, W = img.shape 
    heatmap = Fixpos2Densemap(fix_arr, W, H)#, 0.7, 5)
    print('output', (os.path.join(p_outdir, 'predict_spatial_heatmap_'+str(i)+'.png')))
    cv2.imwrite(os.path.join(p_outdir,'predict_spatial_heatmap_'+str(i)+'.png'), heatmap)
  

  0%|          | 1/326 [00:00<00:54,  5.96it/s]

prediction: /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_prediction/predict_spatial_task_0.png


100%|██████████| 326/326 [00:42<00:00,  7.61it/s]

output /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_predict_heatmaps/predict_spatial_heatmap_0.png


In [20]:
####### Generate heatmaps for ground truth data ######

gt_imdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/'
gt_outdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/'

In [24]:
for i in range(0,13260,340):

  
    gtfile = 'true_spatial_task_'+str(i)+'.png'
     
    gtpath = os.path.join(gt_imdir, gtfile)
    print('ground_truth:',gtpath)
    

    img = cv2.imread(gtpath, cv2.IMREAD_GRAYSCALE)
    result = np.where(img == 255)
#     print('result',result)
    # y = np.array(result[0])[::-1]
    y = np.array(result[0])
    x = np.array(result[1])
    z = np.ones(x.shape)
    fix_arr = np.column_stack([x, y, z])
    H, W = img.shape 
    heatmap = Fixpos2Densemap(fix_arr, W, H)#, 0.7, 5)
    print('output', (os.path.join(gt_outdir, 'true_spatial_heatmap_'+str(i)+'.png')))
    cv2.imwrite(os.path.join(gt_outdir,'true_spatial_heatmap_'+str(i)+'.png'), heatmap)

  0%|          | 1/339 [00:00<00:51,  6.60it/s]

ground_truth: /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_GT/true_spatial_task_0.png


100%|██████████| 339/339 [00:44<00:00,  7.63it/s]

output /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_GT_heatmaps/true_spatial_heatmap_0.png


In [28]:
############### Generate heatmap for mouse data ###########

m_imdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_Mouse/'
m_outdir = '/home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/'


In [31]:
for i in range(0,13260,340):

    mfile = 'mouse_spatial_task_'+str(i)+'.png'
   
    mopath = os.path.join(m_imdir, mfile)
    print('mouse:',mopath)

    img = cv2.imread(mopath, cv2.IMREAD_GRAYSCALE)
    result = np.where(img == 255)
    # y = np.array(result[0])[::-1]
    y = np.array(result[0])
    x = np.array(result[1])
    z = np.ones(x.shape)
    fix_arr = np.column_stack([x, y, z])
    H, W = img.shape 
    heatmap = Fixpos2Densemap(fix_arr, W, H)#, 0.7, 5)
    print('output', (os.path.join(m_outdir, mfile)))
    cv2.imwrite(os.path.join(m_outdir,mfile), heatmap)



  0%|          | 0/145 [00:00<?, ?it/s]

  1%|          | 1/145 [00:00<00:22,  6.32it/s]

mouse: /home/kkjk/LSTM/Images_to_Evaluate/Spatial_Images/sp_Mouse/mouse_spatial_task_0.png




  1%|▏         | 2/145 [00:00<00:21,  6.60it/s]

  2%|▏         | 3/145 [00:00<00:20,  6.80it/s]

  3%|▎         | 4/145 [00:00<00:20,  7.02it/s]

  3%|▎         | 5/145 [00:00<00:19,  7.19it/s]

  4%|▍         | 6/145 [00:00<00:19,  7.25it/s]

  5%|▍         | 7/145 [00:00<00:18,  7.35it/s]

  6%|▌         | 8/145 [00:01<00:18,  7.37it/s]

  6%|▌         | 9/145 [00:01<00:18,  7.39it/s]

  7%|▋         | 10/145 [00:01<00:18,  7.32it/s]

  8%|▊         | 11/145 [00:01<00:18,  7.32it/s]

  8%|▊         | 12/145 [00:01<00:18,  7.38it/s]

  9%|▉         | 13/145 [00:01<00:17,  7.44it/s]

 10%|▉         | 14/145 [00:01<00:17,  7.49it/s]

 10%|█         | 15/145 [00:02<00:17,  7.48it/s]

 11%|█         | 16/145 [00:02<00:17,  7.48it/s]

 12%|█▏        | 17/145 [00:02<00:17,  7.49it/s]

 12%|█▏        | 18/145 [00:02<00:16,  7.50it/s]

 13%|█▎        | 19/145 [00:02<00:16,  7.50it/s]

 14%|█▍        | 20/145 [00:02<00:16,  7.42it/s]

 14%|█▍        | 21/145 [00:02<00:16,  7.43it/s]

 15%|█

output /home/kkjk/LSTM/Images_to_Evaluate/Spatial_heatmaps/sp_Mouse_heatmaps/mouse_spatial_task_0.png


In [32]:
### EOF ###